In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This will likely need to change everythime if your path looks different to mine... thats annoying

In [11]:
PATH_TO_PROJECT = 'drive/Othercomputers/My MacBook Pro/FAERS-analytics'

In [12]:
%cd $PATH_TO_PROJECT

/content/drive/Othercomputers/My MacBook Pro/FAERS-analytics


In [13]:
!ls

'FAERS analytics.ipynb'  'FAERS ASCII Q4 2023'	'FAERS XML Q4 2023'
'FAERS ASCII Q3 2023'	 'FAERS XML Q3 2023'	 README.md
